# Analysis of my 5 favorite stocks

In [1]:
# import libraries 

import pandas as pd
import yfinance as yf
from datetime import datetime

In [45]:
# create start and end date

start_date = datetime.now() - pd.DateOffset(months = 6)
end_date = datetime.now()

In [46]:
# MY FAVORITES :)

tickers = ['SPY', 'QQQ', 'AAPL', 'AMD', 'SHOP']

In [71]:
# download data for the last 3 months from yfinance

df_list = [] # start with an empty list

for ticker in tickers: # iterates through each ticker symbol 
    data = yf.download(ticker, start=start_date, end=end_date) # downloads historical stock data
    data['DailyRange'] = data['High'] - data['Low']
    df_list.append(data) # append list and data
    
df = pd.concat(df_list, keys=tickers, names=['Ticker', 'Date']) # concat list into df

#print
print(df.head())

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
                         Open        High         Low       Close   Adj Close  \
Ticker Date                                                                     
SPY    2023-06-22  433.950012  436.619995  433.600006  436.510010  433.219269   
       2023-06-23  432.929993  435.059998  432.470001  433.209991  429.944122   
       2023-06-26  432.619995  434.609985  431.190002  431.440002  428.187469   
       2023-06-27  432.350006  436.809998  431.880005  436.170013  432.881836   
       2023-06-28  435.049988  437.440002  434.410004  436.390015  433.100189   

                     Volume  DailyRange  
Ticker Date                              
SPY

In [72]:
# reset index from Date col

df = df.reset_index()
print(df.head())

  Ticker       Date        Open        High         Low       Close  \
0    SPY 2023-06-22  433.950012  436.619995  433.600006  436.510010   
1    SPY 2023-06-23  432.929993  435.059998  432.470001  433.209991   
2    SPY 2023-06-26  432.619995  434.609985  431.190002  431.440002   
3    SPY 2023-06-27  432.350006  436.809998  431.880005  436.170013   
4    SPY 2023-06-28  435.049988  437.440002  434.410004  436.390015   

    Adj Close    Volume  DailyRange  
0  433.219269  70637200    3.019989  
1  429.944122  92074500    2.589996  
2  428.187469  72823600    3.419983  
3  432.881836  72813700    4.929993  
4  433.100189  75636000    3.029999  


In [73]:
# visualize stock performance

import plotly.express as px
fig = px.line(df, x='Date', y='Close', color='Ticker', title='Stock Market Performance (6M)')

fig.show()

In [74]:
# facet area chart

fig = px.area(df, x='Date',y='Close', color='Ticker', 
             facet_col = 'Ticker', 
             labels = {'Date':'Date', 'Close':'Closing Price', 'Ticker':'Company'},
             title = 'Stock Prices (6M)')

fig.show()

# Moving Averages

In [75]:
# create moving averages (8 & 21)

df['MA8'] = df.groupby('Ticker')['Close'].rolling(window=8).mean().reset_index(0, drop=True)
df['MA21'] = df.groupby('Ticker')['Close'].rolling(window=21).mean().reset_index(0, drop=True)

# analyze moving averages 

for ticker, group in df.groupby('Ticker'):
    print(f'Moving Averages for {ticker}')
    print(group[['MA8', 'MA21']])

Moving Averages for AAPL
            MA8        MA21
256         NaN         NaN
257         NaN         NaN
258         NaN         NaN
259         NaN         NaN
260         NaN         NaN
..          ...         ...
379  195.478754  192.390477
380  195.925003  192.684762
381  196.258753  193.030000
382  196.148752  193.190953
383  196.336252  193.383333

[128 rows x 2 columns]
Moving Averages for AMD
            MA8        MA21
384         NaN         NaN
385         NaN         NaN
386         NaN         NaN
387         NaN         NaN
388         NaN         NaN
..          ...         ...
507  132.683749  125.497143
508  135.443748  126.405238
509  136.916248  127.335237
510  137.734999  127.999047
511  138.422499  128.987142

[128 rows x 2 columns]
Moving Averages for QQQ
            MA8        MA21
128         NaN         NaN
129         NaN         NaN
130         NaN         NaN
131         NaN         NaN
132         NaN         NaN
..          ...         ...
251  396.78

In [54]:
# visualize moving averages 

for ticker, group in df.groupby('Ticker'):
    fig = px.line(group, x='Date', y=['Close', 'MA8', 'MA21'],
                 title= f'{ticker} Moving Averages')
    fig.show()

# Volatility

In [55]:
# create volatility df

df['Volatility'] = df.groupby('Ticker')['Close'].pct_change().rolling(window=8).std().reset_index(0, drop=True)

#visualize
fig = px.line(df, x='Date', y='Volatility', color='Ticker', title='Volatility of My Top 5')
fig.show()

# Correlation

In [62]:
 import statsmodels.api as sm

In [63]:
# create dataframe with AAPL & QQQ
apple = df.loc[df['Ticker'] == 'AAPL', ['Date', 'Close']].rename(columns={'Close':'AAPL'})
qqq = df.loc[df['Ticker'] == 'QQQ', ['Date', 'Close']].rename(columns={'Close':'QQQ'})
df_corr = pd.merge(apple, qqq, on= 'Date')

#create scatterplot to visualize

fig = px.scatter(df_corr, x='AAPL', y='QQQ', trendline='ols', title='Correlation between AAPL & QQQ')
fig.show()

In [64]:
# create dataframe with AMD & QQQ
amd = df.loc[df['Ticker'] == 'AMD', ['Date', 'Close']].rename(columns={'Close':'AMD'})
qqq = df.loc[df['Ticker'] == 'QQQ', ['Date', 'Close']].rename(columns={'Close':'QQQ'})
df_corr = pd.merge(amd, qqq, on= 'Date')

#create scatterplot to visualize

fig = px.scatter(df_corr, x='AMD', y='QQQ', trendline='ols', title='Correlation between AMD & QQQ')
fig.show()

In [65]:
# create dataframe with AMD & SPY
amd = df.loc[df['Ticker'] == 'AMD', ['Date', 'Close']].rename(columns={'Close':'AMD'})
spy = df.loc[df['Ticker'] == 'SPY', ['Date', 'Close']].rename(columns={'Close':'SPY'})
df_corr = pd.merge(amd, spy, on= 'Date')

#create scatterplot to visualize

fig = px.scatter(df_corr, x='AMD', y='SPY', trendline='ols', title='Correlation between AMD & SPY')
fig.show()

In [66]:
# create dataframe with SHOP & QQQ
shop = df.loc[df['Ticker'] == 'SHOP', ['Date', 'Close']].rename(columns={'Close':'SHOP'})
qqq = df.loc[df['Ticker'] == 'QQQ', ['Date', 'Close']].rename(columns={'Close':'QQQ'})
df_corr = pd.merge(shop, qqq, on= 'Date')

#create scatterplot to visualize

fig = px.scatter(df_corr, x='SHOP', y='QQQ', trendline='ols', title='Correlation between SHOP & QQQ')
fig.show()

In [67]:
# create dataframe with SHOP & SPY
shop = df.loc[df['Ticker'] == 'SHOP', ['Date', 'Close']].rename(columns={'Close':'SHOP'})
spy = df.loc[df['Ticker'] == 'SPY', ['Date', 'Close']].rename(columns={'Close':'SPY'})
df_corr = pd.merge(shop, spy, on= 'Date')

#create scatterplot to visualize

fig = px.scatter(df_corr, x='SHOP', y='SPY', trendline='ols', title='Correlation between SHOP & SPY')
fig.show()

In [68]:
# create dataframe with QQQ & SPY
qqq = df.loc[df['Ticker'] == 'QQQ', ['Date', 'Close']].rename(columns={'Close':'QQQ'})
spy = df.loc[df['Ticker'] == 'SPY', ['Date', 'Close']].rename(columns={'Close':'SPY'})
df_corr = pd.merge(qqq, spy, on= 'Date')

#create scatterplot to visualize

fig = px.scatter(df_corr, x='QQQ', y='SPY', trendline='ols', title='Correlation between QQQ & SPY')
fig.show()

# Daily Range

In [76]:
for ticker, group in df.groupby('Ticker'):
    # reset index to include data column
    group.reset_index(inplace=True)
    #plot daily range for each stock
    fig = px.line(group, x='Date', y='DailyRange',
                 title= f'{ticker} Daily Range')
    fig.show()